In [1]:
%matplotlib inline

In [1]:
import tensorflow as tf
from keras import backend as K
from PIL import Image #winでは無し
from keras.preprocessing import image
#winでは無し
#config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
#sess = tf.Session(config=config)
#K.set_session(sess)

Using TensorFlow backend.


In [3]:
from keras.applications.vgg16 import VGG16

vgg16 = VGG16(include_top=False, weights = None, input_shape=(224, 224, 3))

vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD
#tensorflow.python.kerasが元

def build_transfer_model(vgg16):
    model =  Sequential(vgg16.layers)
     
    #for layer in model.layers[:15]:
    #       layer.trainable = False
            
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7,activation = 'sigmoid')) #number should be changed depend on the categories
        
    return model
        
model = build_transfer_model(vgg16)

In [5]:

model.compile(
    loss = 'categorical_crossentropy',
    optimizer=SGD(lr=0.01, momentum=0.9),
    #optimizer=Adam(lr=0.001),  
    metrics = ['accuracy']
)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
from keras.preprocessing.image import ImageDataGenerator
#from tensorflow.python.keras.applications.vgg16 import preprocess_input


idg_train = ImageDataGenerator(
    rescale=1/255,
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=90,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=True,
    #preprocessing_function=preprocess_input
)

idg_validation = ImageDataGenerator(
    rescale=1/255
)

In [8]:
classes=['-3','-2','-1','0','1','2','3']

img_itr_train= idg_train.flow_from_directory(
    '/Users/ohta/Documents/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/train',
    target_size=(224, 224),
    batch_size=16,
    classes=classes,
    class_mode='categorical',
    shuffle=True
)
img_itr_validation = idg_validation.flow_from_directory(
    '/Users/ohta/Documents/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/test',
    target_size=(224, 224),
    batch_size=16,
    classes=classes,
    class_mode='categorical'
)

Found 1114 images belonging to 7 classes.
Found 160 images belonging to 7 classes.


In [9]:
import os
from datetime import datetime
model_dir = os.path.join(
    'models',
    datetime.now().strftime('%y%m%d_%H%M')
)

os.makedirs(model_dir, exist_ok=True)
print('model_dir:', model_dir) # directory name to save
            
dir_weights = os.path.join(model_dir, 'weights')
os.makedirs(dir_weights, exist_ok=True)
            

model_dir: models/190128_1317


In [10]:
import json
import pickle

model_json = os.path.join(model_dir, 'model.json')
with open(model_json, 'w') as f:
    json.dump(model.to_json(),f)
        
model_classes = os.path.join(model_dir, 'classes.pkl')
with open(model_classes, 'wb') as f:
    pickle.dump(img_itr_train.class_indices, f)

In [11]:
import math

batch_size = 16
steps_per_epoch = math.ceil(
    img_itr_train.samples/batch_size
)

validation_steps = math.ceil(
    img_itr_validation.samples/batch_size
)
print(steps_per_epoch,validation_steps)

70 10


In [12]:
from keras.callbacks import ModelCheckpoint, CSVLogger

cp_filepath = os.path.join(dir_weights, 'ep_{epoch:02d}_ls_{loss:.1f}.h5')
cp = ModelCheckpoint(
                    cp_filepath,
                    monitor='val_acc',
                    verbose=0,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='max',
                    period =5
                    )

csv_filepath = os.path.join(model_dir, 'loss.csv')
csv = CSVLogger(csv_filepath, append=True)

In [13]:
n_epoch = 10

history = model.fit_generator(
    img_itr_train,
    steps_per_epoch=steps_per_epoch,
    epochs=n_epoch,
    validation_data=img_itr_validation,
    validation_steps=validation_steps,
    callbacks = [cp, csv]
    
)

model.save(os.path.join(model_dir,'my_model.h5'))

Epoch 1/10
70/70 [==============================] - 3373s 48s/step - loss: 1.9464 - acc: 0.1202 - val_loss: 1.9460 - val_acc: 0.1437
Epoch 2/10
70/70 [==============================] - 3019s 43s/step - loss: 1.9466 - acc: 0.1412 - val_loss: 1.9459 - val_acc: 0.1437
Epoch 3/10
54/70 [======================>.......] - ETA: 10:49 - loss: 1.9464 - acc: 0.1245

StopIteration: [Errno 23] Too many open files in system: '/Users/ohta/Documents/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/train/-1/0083_20171012_1598764_00000009.png'

In [ ]:
from utils import load_random_imgs
from PIL import Image
import glob

test_data_dir= ''
folders=sorted(glob.glob(test_data_dir+'/**', recursive=False))
files=sorted(glob.glob(folder+'/**.png', recursive=False))

X_test, true_labels = load_random_imgs(
    test_data_dir,
    seed=1
)
x_test=preproc= preprocess_input(x_test.copy())/255.
probs= model.predict(X_test_preproc)

image_size =224
model = load_model(args.model_file)

image = Image.open(args.src_file)
image = image.convert("RGB").resize((image_size, image_size), Image.ANTIALIAS)
image = np.asarray(image, dtype=np.float32).reshape(image_size, image_size, 3)
image /= 255.0

result = model.predict(np.array([image]), 1)[0]
for i, score in enumerate(result):
    print("Category-%d: %.2f"%(i, score))
    
    
# 学習済みのVGG16をロード
# 構造とともに学習済みの重みも読み込まれる
model = VGG16(weights='imagenet')
# model.summary()

stack=[]
predict_result=[]

for file in files:
# 引数で指定した画像ファイルを読み込む
# サイズはVGG16のデフォルトである224x224にリサイズされる
    img = image.load_img(file, target_size=(224, 224))

# 読み込んだPIL形式の画像をarrayに変換
    x = image.img_to_array(img)
    x=preprocess_input(x)
    # 3次元テンソル（rows, cols, channels) を
    # 4次元テンソル (samples, rows, cols, channels) に変換
    # 入力画像は1枚なのでsamples=1でよい
    x = np.expand_dims(x, axis=0)

    
    # Top-5のクラスを予測する
    # VGG16の1000クラスはdecode_predictions()で文字列に変換される
    preds = model.predict(preprocess_input(x))
    results = decode_predictions(preds, top=5)[0]
    for result in results:
        print(result)    
        
    

In [12]:
#画像全体を配列に変換してから予測する
#入力ファイルタイプに注意jpg png
import keras
import os
import glob
import numpy as np
from PIL import Image
from keras.utils import np_utils
#from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
#from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array,  load_img
#list_pictures,
from sklearn.metrics import confusion_matrix


#ラベル毎にフォルダ分けされた親フォルダ名   ーーー pngだとエラー 
path='/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina'
#path='/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TItest'
folders=sorted(glob.glob(path+'/**', recursive=False))
category_number=len(folders)
print('folders',folders)
model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TIscout_model/TI_DNN_20172018_acc069.h5')#モデルも保存する設定にしてあるh5
#model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TIscout_model/TI_20172018.h5')

X =[]
Y = []
Fnames=[]
size=224 #変換先の画像サイズ
categoryname=[]
teacher_category=[]

for category_num, folder in enumerate(folders):
    categoryname.append(os.path.basename(folder))
    print('category number',category_num,folder)
    
    files=sorted(glob.glob(folder+"/*JPG")) #
    print('detected files',files)
    for filepath in files:
        #print(filepath)
        #img = img_to_array(load_img(filepath, target_size=(size,size))) #color_mode外した
        img=Image.open(filepath)
        img=img.convert("RGB")
        img=img.resize((size,size))
        img=np.asarray(img)
        X.append(img)
        Y.append(category_num) # 教師データ（正解）
        teacher_category.append(os.path.basename(folder))
        Fnames.append(os.path.basename(filepath))
    
    # 学習データの取得（非正解画像）
    #for filename in list_pictures('./neg/'):
        #img = img_to_array(load_img(filepath, target_size=(64,64)))
        #X.append(img)
        #Y.append(1) # 教師データ（正解）
print(Fnames)
    # NumPy配列に変換
X = np.asarray(X)
Y = np.asarray(Y)
    # float32型に変換
    # 正規化(0～1)   
X = X.astype('float32')/255
Y = Y.astype('int32')
#X=X.respahe(-1,size,size,3)
# クラスの形式を変換
Y = np_utils.to_categorical(Y, category_number)
print(X.shape)    

#予測
print('predicting')
probs=model.predict(X, verbose=1)
label=probs.argmax(axis=1)


#confusion matrixで表示 sklernを使用
#縦が実際の分類結果、横が予測結果なので通常の2x2と逆
predict_classes = model.predict_classes(X, batch_size=32)
true_classes = np.argmax(Y,1)
print(confusion_matrix(true_classes, predict_classes))

print("completed")


folders ['/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/0a3', '/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/0b2', '/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/0c1', '/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/a0', '/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/a1', '/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/a2', '/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/a3']
category number 0 /Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina/0a3
detected files ['/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_t

In [10]:
#probsから正解ラベルを抽出、ファイル名と共に書き出し

import pandas as pd
#pngファイルはダメみたい
#path='/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TItest'
path='/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/TIdata20172018/TIscout_test_retina'
folders=sorted(glob.glob(path+'/**', recursive=False))

categoryclass=[]
for category_num, folder in enumerate(folders):
    categoryclass.append(os.path.basename(folder))  
    
labels=probs.argmax(axis=1)
df = pd.DataFrame(probs,columns=categoryclass)
df['result_index']=labels
df['teacher_index']=teacher_category
df['filename']=Fnames

print(df)

df.to_csv(path+'DNNresult_TIscout20172018by20172018learn_all_retina.csv', header=True, index=False)



              0a3           0b2           0c1            a0            a1  \
0    9.793448e-01  2.055427e-02  1.032781e-05  6.534941e-06  6.277101e-05   
1    9.999204e-01  5.814256e-05  4.307297e-11  4.924441e-10  3.159518e-08   
2    9.862976e-01  1.345943e-02  1.568753e-05  7.035851e-06  1.255183e-04   
3    8.492852e-01  1.500646e-01  5.178586e-04  2.808876e-05  7.026989e-05   
4    3.824700e-02  9.556587e-01  6.089771e-03  3.273647e-06  7.356696e-07   
5    2.566322e-01  6.628829e-01  7.810728e-02  1.234138e-03  8.377423e-04   
6    4.528973e-01  5.052047e-01  4.038582e-02  1.101414e-03  3.691596e-04   
7    3.138187e-01  5.846501e-01  9.915841e-02  1.731769e-03  5.613135e-04   
8    6.909767e-05  1.293889e-04  1.942453e-02  5.431578e-01  4.303240e-01   
9    9.999996e-01  3.345550e-07  2.288456e-14  3.108853e-12  1.318451e-09   
10   9.999998e-01  2.236222e-07  6.133109e-14  1.311753e-11  1.760965e-08   
11   9.996650e-01  5.371984e-05  3.198732e-08  1.151244e-06  2.368653e-04   

In [14]:
#print(categoryclass)

['0a3', '0b2', '0c1', 'a0', 'a1', 'a2', 'a3']


In [6]:
#1枚のファイルを読んで表示

import keras
import os
import glob
import numpy as np
from PIL import Image
from keras.utils import np_utils
#from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
#from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array,  load_img #, list_pictures
from sklearn.metrics import confusion_matrix

imgfile='/Users/ohta/Downloads/IMG_6397.JPG'

model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TIscout_model/TI_DNN_20172018_acc069.h5')
#model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TInull_182case_m3top3/ep_35_ls_1.1.h5')
#model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TIscout_model/TI_20172018.h5')#モデルも保存する設定にしてあるh5
#model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TIscout_model/TIscout_432_Ohta7_model.h5')


image_size=224

image = Image.open(imgfile)
image = image.convert("RGB").resize((image_size, image_size), Image.ANTIALIAS)
image = np.asarray(image, dtype=np.float32).reshape(image_size, image_size, 3)
image /= 255.0

result = model.predict(np.array([image]), 1)[0]
for i, score in enumerate(result):
    print("Category-%d: %.6f"%(i, score))

Category-0: 0.001487
Category-1: 0.002965
Category-2: 0.168351
Category-3: 0.598857
Category-4: 0.226345
Category-5: 0.001948
Category-6: 0.000047


In [3]:
print('estimated optimum TI-null is:')
print('20msec (@1.5T) or 30msec (@3T)')
print('higher than this TI image')

estimated optimum TI-null is:
20msec (@1.5T) or 30msec (@3T)
higher than this TI image


In [1]:

#1枚のファイルを読んで表示

import keras
import os
import glob
import numpy as np
import cv2
from PIL import Image
from keras.utils import np_utils
#from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
#from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from sklearn.metrics import confusion_matrix

imgfile='/Users/ohta/Downloads/IMG_5248.JPG'

model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TIscout_model/TIscout_432_Ohta7_model.h5')#モデルも保存する設定にしてあるh5


image_size=224# VideoCaptureのインスタンスを作成する。
# 引数でカメラを選べれる。
cap = cv2.VideoCapture(0)

while True:
    # VideoCaptureから1フレーム読み込む
    ret, frame = cap.read()

    # スクリーンショットを撮りたい関係で1/4サイズに縮小
    frame = cv2.resize(frame, (image_size, image_size))
    # 加工なし画像を表示する
    cv2.imshow('Raw Frame', frame)

    image = frame
    #image = image.convert("RGB").resize((image_size, image_size), Image.ANTIALIAS)
    image = np.asarray(image, dtype=np.float32).reshape(image_size, image_size, 3)
    image /= 255.0

    result = model.predict(np.array([image]), 1)[0]
    for i, score in enumerate(result):
        print("Category-%d: %.2f"%(i, score))
    

    # キー入力を1ms待って、k が27（ESC）だったらBreakする
    k = cv2.waitKey(1)
    if k == 27:
        break

# キャプチャをリリースして、ウィンドウをすべて閉じる
cap.release()
cv2.destroyAllWindows()

Using TensorFlow backend.


error: OpenCV(3.4.5) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:3784: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [12]:
cap.release()
cv2.destroyAllWindows()

In [ ]:

#1枚のファイルを読んで表示

import keras
import os
import glob
import numpy as np
import cv2
from PIL import Image
from keras.utils import np_utils
#from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
#from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from sklearn.metrics import confusion_matrix

imgfile='/Users/ohta/Downloads/イメージ.jpg'

model=load_model('/Users/ohta/OneDrive - 国立研究開発法人 国立循環器病研究センター/TIscout_model/TIscout_432_Ohta7_model.h5')
#モデルも保存する設定にしてあるh5


image_size=224# VideoCaptureのインスタンスを作成する。

print('please send image to downloadfolder from your iphone')
print('press ESC key to break')
while True:
    
    if os.path.isfile(imgfile)==True:
        break
  
    # キー入力を1ms待って、k が27（ESC）だったらBreakする
    k = cv2.waitKey(1)
    if k == 27:
        break

image = Image.open(imgfile)
#image = image.convert("RGB").resize((image_size, image_size), Image.ANTIALIAS)
img=img.convert("RGB")
img=img.resize((size,size), Image.ANTIALIAS)
image = np.asarray(image, dtype=np.float32).reshape(image_size, image_size, 3)
image /= 255.0

result = model.predict(np.array([image]), 1)[0]
for i, score in enumerate(result):
    print("Category-%d: %.2f"%(i, score))
        
# キャプチャをリリースして、ウィンドウをすべて閉じる
cap.release()
cv2.destroyAllWindows()

In [3]:
keras -version

NameError: name 'version' is not defined